In [62]:
import h5py as h5
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import sys
sys.path.append(os.path.expanduser("~/TMPredictor/survival_tm/auton-survival"))
from auton_survival.preprocessing import Scaler
import optuna
from sklearn.model_selection import ParameterGrid
#sys.path.append('/projects/EKOLEMEN/survival_tm/train_models/auton-survival')
sys.path.append(os.path.expanduser("~/TMPredictor/survival_tm/auton-survival"))
from sklearn.model_selection import ParameterGrid
from auton_survival.estimators import SurvivalModel
from auton_survival.metrics import survival_regression_metric
from auton_survival.models.dsm import DeepSurvivalMachines
from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc
from sklearn.decomposition import PCA

FROM TM LABELS AND DATABASE, CREATE X, T, E, SHOTS

In [45]:

#database = h5.File('/projects/EKOLEMEN/profile_predictor/joe_hiro_models/diiid_data.h5', 'r')
tm_data = h5.File('/projects/EKOLEMEN/survival_tm/tm_labels.h5', 'r')

signals = ['pinj', 'tinj', 'betan_EFITRT2', 'qmin_EFITRT2', 'ech_pwr_total', 'ip', 'bt', 'li_EFITRT2', 'aminor_EFITRT2', 
          'rmaxis_EFITRT2', 'tribot_EFITRT2', 'tritop_EFITRT2', 'kappa_EFITRT2', 'volume_EFITRT2']

prof_signals = ['thomson_temp_mtanh_1d', 'cer_temp_csaps_1d', 'thomson_density_mtanh_1d', 'cer_rot_csaps_1d', 'qpsi_EFITRT2', 'pres_EFITRT2']

with open('data_dict-2.pkl', 'rb') as f:
    data = pickle.load(f)
# this shot has messed up datastructure
del data['192209']
shots = list(data.keys())
print(data[shots[0]].keys())
def check_all_signals(shot_data, prof_signals, signals, ind):
    # Check scalars for nan values
    for sig in signals:
        if np.isnan(shot_data[sig][ind]):
            return False
    
    # Check profiles for nan values
    for sig in prof_signals:
        if np.sum(np.isnan(shot_data[sig][ind,:])) > 0:
            return False
       
    return True

def gather_feature(shot_data, prof_signals, signals, ind):
    feature = np.zeros(len(prof_signals)*33+len(signals))
    
    # Load scalar signals
    for i, sig in enumerate(signals):
        feature[i] = shot_data[sig][ind]
    
    # Load profile signals
    for i, sig in enumerate(prof_signals):
        feature[33*i+len(signals):33*(i+1)+len(signals)] = shot_data[sig][ind,:]
    
    return feature

x = []
t = []
e = []
shots_rt = []
times = np.arange(0, 6000, 20)[::-1]
# I should check the timesteps are the same for all scalars
for shot in shots:
    if shot in list(tm_data.keys()):
        # First figure out time of TM, or time of end of shot
        label = tm_data[shot]['label'][:]
        time = tm_data[shot]['time'][:]
        tm_ind = np.argmax(label>0)
        tm_time = time[tm_ind]
        # Find start and stop of flattop
        start = np.array(data[shot]['t_ip_flat_sql'])
        duration = np.array(data[shot]['ip_flat_duration_sql'])
        stop = start+duration
        # Get start and stop inds
        start_ind = np.argmin(np.abs(time - start))
        end_ind = np.argmin(np.abs(time - stop))
        
        # End shot when TM occurs
        if end_ind > tm_ind and tm_ind > 0:
            end_ind = tm_ind
        
        # Go through valid indices and gather dataset
        for ind in range(start_ind, end_ind):
            # Check that we have all signals at this time
            if check_all_signals(data[shot], prof_signals, signals, ind):
                feature = gather_feature(data[shot], prof_signals, signals, ind)
                x.append(feature)
                if tm_time > 0:
                    # Time to TM
                    t.append(tm_time - time[ind])
                    shots_rt.append(shot)
                    e.append(1)
                else:
                    # Time to end of shot
                    t.append(time[end_ind] - time[ind])
                    shots_rt.append(shot)
                    e.append(0)

dict_keys(['pinj', 'tinj', 'betan_EFITRT2', 'qmin_EFITRT2', 'ech_pwr_total', 'ip', 'bt', 'li_EFITRT2', 'aminor_EFITRT2', 'rmaxis_EFITRT2', 'tribot_EFITRT2', 'tritop_EFITRT2', 'kappa_EFITRT2', 'volume_EFITRT2', 't_ip_flat_sql', 'ip_flat_duration_sql', 'thomson_temp_mtanh_1d', 'cer_temp_csaps_1d', 'thomson_density_mtanh_1d', 'cer_rot_csaps_1d', 'qpsi_EFITRT2', 'pres_EFITRT2'])


In [168]:
'''with open('data/rt_x.pkl', 'wb') as f:
    pickle.dump(np.array(x), f)
with open('data/rt_t.pkl', 'wb') as f:
    pickle.dump(t, f)
with open('data/rt_e.pkl', 'wb') as f:
    pickle.dump(e, f)
with open('data/rt_shots.pkl', 'wb') as f:
    pickle.dump(shots_rt, f)'''

In [188]:
data = x
mean = np.mean(data, axis=0)
std_dev = np.std(data, axis=0)
mean[2] = 0
std_dev[2] = 1
mean[7] = 0
std_dev[7] = 1

# Define the threshold for filtering
threshold = 8*std_dev
data[:, 14+33*4:14+33*5] = 1/data[:, 14+33*4:14+33*5]
# Identify rows with any values that are more than 10 standard deviations away from the mean for any column
rows_with_anomalies = np.any(np.abs(data - mean) > threshold, axis=1)
data[:, 14+33*4:14+33*5] = 1/data[:, 14+33*4:14+33*5]
# Filter out the rows with anomalies
filtered_data = data[~rows_with_anomalies]

'''with open('data/rt_filtered_x.pkl', 'wb') as f:
    pickle.dump(filtered_data, f)
with open('data/rt_filtered_t.pkl', 'wb') as f:
    pickle.dump(np.array(t)[~rows_with_anomalies], f)
with open('data/rt_filtered_e.pkl', 'wb') as f:
    pickle.dump(np.array(e)[~rows_with_anomalies], f)
with open('data/rt_filtered_shots.pkl', 'wb') as f:
    pickle.dump(np.array(shots_rt)[~rows_with_anomalies], f)'''

/tmp/ipykernel_3615927/4178029613.py:12: RuntimeWarning: divide by zero encountered in divide
  data[:, 14+33*4:14+33*5] = 1/data[:, 14+33*4:14+33*5]


CREATE MAIN PREPROCESSED DATABASE

In [12]:
shots = np.load('/projects/EKOLEMEN/survival_tm/shots.npy')
tm_shots = np.load('/projects/EKOLEMEN/survival_tm/tm_shots.npy')
st_shots = np.load('/projects/EKOLEMEN/survival_tm/st_shots.npy')

def load_data(data_type):
    with open(f'/projects/EKOLEMEN/survival_tm/formatted_labels/{data_type}.pkl', 'rb') as f:
        data = pickle.load(f)
    
    return data['x'], data['t'], data['e']

# Don't want whole dataset right now so 0.5 factor to reduce
n = len(shots)

tr_size = int(n*0.80)
vl_size = int(n*0.10)
te_size = int(n*0.10)

train_shots = shots[:tr_size]
test_shots = shots[-te_size:]
valid_shots = shots[tr_size:tr_size+vl_size]

x_train, t_train, e_train = load_data('train')
x_test,  t_test,  e_test  = load_data('test')
x_valid, t_valid, e_valid = load_data('valid')

# Get inds for time <600ms
# is this needed? Not sure.
'''inds = np.where(t_train < 600)[0]

x_train = x_train[inds]
t_train = t_train[inds]
e_train = e_train[inds]'''

tm_inds = np.where(e_train == 1)[0]
st_inds = np.where(e_train == 0)[0]
new_st_inds = np.random.choice(st_inds, size=len(tm_inds), replace=False)
combined_inds = np.concatenate((tm_inds, new_st_inds))
sorted_combined_inds = np.sort(combined_inds)

x_train = x_train[sorted_combined_inds]
t_train = t_train[sorted_combined_inds]
e_train = e_train[sorted_combined_inds]

'''x_train = np.concatenate((x_train[tm_inds], x_train[new_st_inds]), axis=0)
t_train = np.concatenate((t_train[tm_inds], t_train[new_st_inds]), axis=0)
e_train = np.concatenate((e_train[tm_inds], e_train[new_st_inds]), axis=0)
plt.plot(t_train[10000:11000])
# Shuffle arrays because currently all 1s followed by all 0s
p = np.random.permutation(len(t_train))
x_train = x_train[p,:]
t_train = t_train[p]
e_train = e_train[p]'''

x_train_df = pd.DataFrame(x_train)
t_train_df = pd.DataFrame(t_train)
e_train_df = pd.DataFrame(e_train)

x_valid_df = pd.DataFrame(x_valid)
t_valid_df = pd.DataFrame(t_valid)
e_valid_df = pd.DataFrame(e_valid)

x_test_df = pd.DataFrame(x_test)
t_test_df = pd.DataFrame(t_test)
e_test_df = pd.DataFrame(e_test)

outcomes_valid_df = pd.DataFrame({'time': t_valid, 'event': e_valid})

outcomes_train_df = pd.DataFrame({'time': t_train, 'event': e_train})
outcomes_test_df = pd.DataFrame({'time': t_test, 'event': e_test})
#normalize
'''scaler = Scaler()
transformer = scaler.fit(x_train_df)
x_train_df_normed = transformer.transform(x_train_df)
x_valid_df_normed = transformer.transform(x_valid_df)
x_test_df_normed = transformer.transform(x_test_df)'''
'''
with open('data/x_train_df1.pkl', 'wb') as f:
    pickle.dump(x_train_df, f)
with open('data/x_valid_df1.pkl', 'wb') as f:
    pickle.dump(x_valid_df, f)
with open('data/x_test_df1.pkl', 'wb') as f:
    pickle.dump(x_test_df, f)
with open('data/outcomes_valid_df1.pkl', 'wb') as f:
    pickle.dump(outcomes_valid_df, f)
with open('data/outcomes_train_df1.pkl', 'wb') as f:
    pickle.dump(outcomes_train_df, f)
with open('data/outcomes_test_df1.pkl', 'wb') as f:
    pickle.dump(outcomes_test_df, f)'''

FIND NORMALIZATION (UNUSED, WE NOW NORMALIZE AFTER PCA)

In [6]:
# q will be normalized by inverting rather than normalizing
# TRY NORMALIZING PROFILE BY PROFILE NOT EACH ELEMENT
q_start_index = 146
train_means = np.mean(x_train[:, :q_start_index], axis=0)
train_std = np.std(x_train[:, :q_start_index], axis=0)
valid_means = np.mean(x_valid[:, :q_start_index], axis=0)
valid_std = np.std(x_valid[:, :q_start_index], axis=0)
train_iotas = 1/x_train[:, q_start_index:]
valid_iotas = 1/x_valid[:, q_start_index:]
x_train_norm = (x_train[:, :q_start_index] - train_means)/train_std
x_valid_norm = (x_valid[:, :q_start_index] - valid_means)/valid_std

x_train_norm = np.concatenate((x_train_norm, train_iotas), axis=1)
x_valid_norm = np.concatenate((x_valid_norm, valid_iotas), axis=1)
train_mask = np.all(x_train_norm <= 10, axis=1)
valid_mask = np.all(x_valid_norm <= 10, axis=1)
# Filter the array to keep only rows where all values are <= 10
x_train_norm = pd.DataFrame(x_train_norm[train_mask])
x_valid_norm = pd.DataFrame(x_valid_norm[valid_mask])
t_train = t_train[train_mask]
e_train = e_train[train_mask]
t_valid = t_valid[valid_mask]
e_valid = e_valid[valid_mask]
outcomes_valid_df = pd.DataFrame({'time': t_valid, 'event': e_valid})
outcomes_train_df = pd.DataFrame({'time': t_train, 'event': e_train})

'''with open('data/x_train_manual_norm.pkl', 'wb') as f:
    pickle.dump(x_train_norm, f)
with open('data/x_valid_manual_norm.pkl', 'wb') as f:
    pickle.dump(x_valid_norm, f)

with open('data/outcomes_valid_manual_norm.pkl', 'wb') as f:
    pickle.dump(outcomes_valid_df, f)
with open('data/outcomes_train_manual_norm.pkl', 'wb') as f:
    pickle.dump(outcomes_train_df, f)'''

CREATE PCA DATABASE

In [190]:
'''with open('data/x_train_df1.pkl', 'rb') as f:
    x_train_df = pickle.load(f)
with open('data/x_valid_df1.pkl', 'rb') as f:
    x_valid_df = pickle.load(f)
with open('data/outcomes_train_df1.pkl', 'rb') as f:
    outcomes_train_df = pickle.load(f)
with open('data/outcomes_valid_df1.pkl', 'rb') as f:
    outcomes_valid_df = pickle.load(f)'''

with open('data/rt_filtered_x.pkl', 'rb') as f:
    x_train_df = pickle.load(f)

# separate each normalized profile
scalars = np.array(x_train_df)[:, 0:14]
te = np.array(x_train_df)[:, 14:33+14]
ti = np.array(x_train_df)[:, 33+14:66+14]
ne = np.array(x_train_df)[:, 66+14:99+14]
rot = np.array(x_train_df)[:, 99+14:132+14]
qpsi = np.array(x_train_df)[:, 132+14:165+14]
qpsi = 1/qpsi
qpsi = np.where(qpsi == np.inf, 1, qpsi)
pres = np.array(x_train_df)[:, 165+14:198+14]

# upsample to PCS dimensions
x_old = np.linspace(0, 1, 33)
x_long = np.linspace(0, 1, 100)
x_for_q = np.linspace(0, 1, 65)
te = np.array([np.interp(x_long, x_old, te[i]) for i in range(len(te))])
ti = np.array([np.interp(x_long, x_old, ti[i]) for i in range(len(ti))])
ne = np.array([np.interp(x_long, x_old, ne[i]) for i in range(len(ne))])
rot = np.array([np.interp(x_long, x_old, rot[i]) for i in range(len(rot))])
qpsi = np.array([np.interp(x_for_q, x_old, qpsi[i]) for i in range(len(qpsi))])
pres = np.array([np.interp(x_for_q, x_old, pres[i]) for i in range(len(pres))])

pca_components = []
# Fit PCA on the data
pca = PCA(n_components=4)
pca.fit(te)
te_pca = pca.transform(te)
pca_components.append(pca.components_)
pca_components.append(pca.mean_)

pca = PCA(n_components=4)
pca.fit(ti)
ti_pca = pca.transform(ti)
pca_components.append(pca.components_)
pca_components.append(pca.mean_)

pca = PCA(n_components=4)
pca.fit(ne)
ne_pca = pca.transform(ne)
pca_components.append(pca.components_)
pca_components.append(pca.mean_)

pca = PCA(n_components=4)
pca.fit(rot)
rot_pca = pca.transform(rot)
pca_components.append(pca.components_)
pca_components.append(pca.mean_)

pca = PCA(n_components=4)
pca.fit(qpsi)
qpsi_pca = pca.transform(qpsi)
pca_components.append(pca.components_)
pca_components.append(pca.mean_)

pca = PCA(n_components=4)
pca.fit(pres)
pres_pca = pca.transform(pres)
pca_components.append(pca.components_)
pca_components.append(pca.mean_)
# normalize PCA components

scalars_mean = np.mean(scalars, axis=0)
scalars_std = np.std(scalars, axis=0)
scalars_normed = (scalars - scalars_mean)/scalars_std

te_pca_mean = np.mean(te_pca)
te_pca_std = np.std(te_pca)
te_pca_normed = (te_pca - te_pca_mean)/te_pca_std

ti_pca_mean = np.mean(ti_pca)
ti_pca_std = np.std(ti_pca)
ti_pca_normed = (ti_pca - ti_pca_mean)/ti_pca_std

ne_pca_mean = np.mean(ne_pca)
ne_pca_std = np.std(ne_pca)
ne_pca_normed = (ne_pca - ne_pca_mean)/ne_pca_std

rot_pca_mean = np.mean(rot_pca)
rot_pca_std = np.std(rot_pca)
rot_pca_normed = (rot_pca - rot_pca_mean)/rot_pca_std

qpsi_pca_mean = np.mean(qpsi_pca)
qpsi_pca_std = np.std(qpsi_pca)
qpsi_pca_normed = (qpsi_pca - qpsi_pca_mean)/qpsi_pca_std

pres_pca_mean = np.mean(pres_pca)
pres_pca_std = np.std(pres_pca)
pres_pca_normed = (pres_pca - pres_pca_mean)/pres_pca_std

normalizations = [(scalars_mean, scalars_std), (te_pca_mean, te_pca_std), (ti_pca_mean, ti_pca_std), (ne_pca_mean, ne_pca_std), (rot_pca_mean, rot_pca_std), (qpsi_pca_mean, qpsi_pca_std), (pres_pca_mean, pres_pca_std)]

x_train_pca = pd.DataFrame(np.concatenate((scalars_normed, te_pca_normed, ti_pca_normed, ne_pca_normed, rot_pca_normed, qpsi_pca_normed, pres_pca_normed), axis=1))
'''
with open('data/rt_x_pca.pkl', 'wb') as f:
    pickle.dump(x_train_pca, f)

with open('data/rt_normalizations.pkl', 'wb') as f:
    pickle.dump(normalizations, f)

with open('data/rt_pca_components.pkl', 'wb') as f:
    pickle.dump(pca_components, f)'''

In [5]:
def find_peaks_in_data(data):
    peaks = []
    for i in range(1, len(data) - 1):
        if data[i-1] < data[i] > data[i+1]:
            peaks.append(i)
    return peaks

CREATE MEMORY DATABASE

In [6]:
# we give each timestep all the info from a few timesteps in the past. 

x_train_np = np.array(x_train)
t_train_np = np.array(t_train)[4:]
e_train_np = np.array(e_train)[4:]

x_valid_np = np.array(x_valid)
t_valid_np = np.array(t_valid)[4:]
e_valid_np = np.array(e_valid)[4:]

x_test_np = np.array(x_test)
t_test_np = np.array(t_test)[4:]
e_test_np = np.array(e_test)[4:]

outcomes_valid_memory = pd.DataFrame({'time': t_valid_np, 'event': e_valid_np})
outcomes_test_memory = pd.DataFrame({'time': t_test_np, 'event': e_test_np})
outcomes_train_memory = pd.DataFrame({'time': t_train_np, 'event': e_train_np})

# make the state of each timestep contain 5 timesteps of data
x_valid_memory = np.array([x_valid_np[i-4:i+1].flatten() for i in range(4, len(x_valid_np))])
x_test_memory = np.array([x_test_np[i-4:i+1].flatten() for i in range(4, len(x_test_np))])
x_train_memory = np.array([x_train_np[i-4:i+1].flatten() for i in range(4, len(x_train_np))])

# remove the first 4 timesteps of each shot from the dataset. This is so that data from the previous shot isn't saved in the current shot
train_peaks = np.array(find_peaks_in_data(np.array(outcomes_train_memory['time'])))
valid_peaks = np.array(find_peaks_in_data(np.array(outcomes_valid_memory['time'])))
test_peaks = np.array(find_peaks_in_data(np.array(outcomes_test_memory['time'])))
new_train_peaks = np.sort(np.concatenate((train_peaks, train_peaks+1, train_peaks+2, train_peaks+3)))
new_valid_peaks = np.sort(np.concatenate((valid_peaks, valid_peaks+1, valid_peaks+2, valid_peaks+3)))
new_test_peaks = np.sort(np.concatenate((test_peaks, test_peaks+1, test_peaks+2, test_peaks+3)))
x_valid_memory = pd.DataFrame(np.delete(x_valid_memory, new_valid_peaks, axis=0))
x_test_memory = pd.DataFrame(np.delete(x_test_memory, new_test_peaks, axis=0))
x_train_memory = pd.DataFrame(np.delete(x_train_memory, new_train_peaks, axis=0))

# normalize
scaler = Scaler()
transformer = scaler.fit(x_train_memory)
x_train_memory_normed = transformer.transform(x_train_memory)
x_valid_memory_normed = transformer.transform(x_valid_memory)
x_test_memory_normed = transformer.transform(x_test_memory)

'''with open('data/x_valid_memory_normed.pkl', 'wb') as f:
    pickle.dump(x_valid_memory_normed, f)
with open('data/x_test_memory_normed.pkl', 'wb') as f:
    pickle.dump(x_test_memory_normed, f)
with open('data/x_train_memory_normed.pkl', 'wb') as f:
    pickle.dump(x_train_memory_normed, f)
with open('data/outcomes_valid_memory.pkl', 'wb') as f:
    pickle.dump(outcomes_valid_memory, f)
with open('data/outcomes_train_memory.pkl', 'wb') as f:
    pickle.dump(outcomes_train_memory, f)
with open('data/outcomes_test_memory.pkl', 'wb') as f:
    pickle.dump(outcomes_test_memory, f)'''

"with open('data/x_valid_memory_normed.pkl', 'wb') as f:\n    pickle.dump(x_valid_memory_normed, f)\nwith open('data/x_test_memory_normed.pkl', 'wb') as f:\n    pickle.dump(x_test_memory_normed, f)\nwith open('data/x_train_memory_normed.pkl', 'wb') as f:\n    pickle.dump(x_train_memory_normed, f)\nwith open('data/outcomes_valid_memory.pkl', 'wb') as f:\n    pickle.dump(outcomes_valid_memory, f)\nwith open('data/outcomes_train_memory.pkl', 'wb') as f:\n    pickle.dump(outcomes_train_memory, f)\nwith open('data/outcomes_test_memory.pkl', 'wb') as f:\n    pickle.dump(outcomes_test_memory, f)"

CREATE FUTURE ACTUATORS DATABASE. SAME LOGIC AS ABOVE, BUT ONLY SAVING SCALARS DATA

In [7]:
signals = ['pinj', 'tinj', 'betan_EFIT01', 'qmin_EFIT01', 'ech_pwr_total', 'ip', 'bt', 'li_EFIT01', 'aminor_EFIT01', 
          'rmaxis_EFIT01', 'tribot_EFIT01', 'tritop_EFIT01', 'kappa_EFIT01', 'volume_EFIT01']

# we give each timestep all the info from a few timesteps in the past. 

x_train_np = np.array(x_train)
t_train_np = np.array(t_train)[:-4]
e_train_np = np.array(e_train)[:-4]

x_valid_np = np.array(x_valid)
t_valid_np = np.array(t_valid)[:-4]
e_valid_np = np.array(e_valid)[:-4]

x_test_np = np.array(x_test)
t_test_np = np.array(t_test)[:-4]
e_test_np = np.array(e_test)[:-4]

outcomes_valid_future = pd.DataFrame({'time': t_valid_np, 'event': e_valid_np})
outcomes_test_future = pd.DataFrame({'time': t_test_np, 'event': e_test_np})
outcomes_train_future = pd.DataFrame({'time': t_train_np, 'event': e_train_np})

# we keep the present and 4 in the future
x_valid_future = pd.DataFrame([np.concatenate((x_valid_np[i:i+5, 0:len(signals)].flatten(), x_valid_np[i, len(signals):])) for i in range(len(x_valid_np)-4)])
x_test_future = pd.DataFrame([np.concatenate((x_test_np[i:i+5, 0:len(signals)].flatten(), x_test_np[i, len(signals):])) for i in range(len(x_test_np)-4)])
x_train_future = pd.DataFrame([np.concatenate((x_train_np[i:i+5, 0:len(signals)].flatten(), x_train_np[i, len(signals):])) for i in range(len(x_train_np)-4)])

train_peaks = np.array(find_peaks_in_data(np.array(outcomes_train_future['time'])))
valid_peaks = np.array(find_peaks_in_data(np.array(outcomes_valid_future['time'])))
test_peaks = np.array(find_peaks_in_data(np.array(outcomes_test_future['time'])))

new_train_peaks = np.sort(np.concatenate((train_peaks-1, train_peaks-2, train_peaks-3, train_peaks-4)))
new_valid_peaks = np.sort(np.concatenate((valid_peaks-1, valid_peaks-2, valid_peaks-3, valid_peaks-4)))
new_test_peaks = np.sort(np.concatenate((test_peaks-1, test_peaks-2, test_peaks-3, test_peaks-4)))

x_valid_future = pd.DataFrame(np.delete(x_valid_future, new_valid_peaks, axis=0))
x_test_future = pd.DataFrame(np.delete(x_test_future, new_test_peaks, axis=0))
x_train_future = pd.DataFrame(np.delete(x_train_future, new_train_peaks, axis=0))

scaler = Scaler()
transformer = scaler.fit(x_train_future)
x_train_future_normed = transformer.transform(x_train_future)
x_valid_future_normed = transformer.transform(x_valid_future)
x_test_future_normed = transformer.transform(x_test_future)

'''with open('data/x_valid_future_normed.pkl', 'wb') as f:
    pickle.dump(x_valid_future_normed, f)
with open('data/x_test_future_normed.pkl', 'wb') as f:
    pickle.dump(x_test_future_normed, f)
with open('data/x_train_future_normed.pkl', 'wb') as f:
    pickle.dump(x_train_future_normed, f)
with open('data/outcomes_valid_future.pkl', 'wb') as f:
    pickle.dump(outcomes_valid_future, f)
with open('data/outcomes_train_future.pkl', 'wb') as f:
    pickle.dump(outcomes_train_future, f)
with open('data/outcomes_test_future.pkl', 'wb') as f:
    pickle.dump(outcomes_test_future, f)'''

"with open('data/x_valid_future_normed.pkl', 'wb') as f:\n    pickle.dump(x_valid_future_normed, f)\nwith open('data/x_test_future_normed.pkl', 'wb') as f:\n    pickle.dump(x_test_future_normed, f)\nwith open('data/x_train_future_normed.pkl', 'wb') as f:\n    pickle.dump(x_train_future_normed, f)\nwith open('data/outcomes_valid_future.pkl', 'wb') as f:\n    pickle.dump(outcomes_valid_future, f)\nwith open('data/outcomes_train_future.pkl', 'wb') as f:\n    pickle.dump(outcomes_train_future, f)\nwith open('data/outcomes_test_future.pkl', 'wb') as f:\n    pickle.dump(outcomes_test_future, f)"